In [105]:
import pandas
import numpy
from scipy.interpolate import UnivariateSpline

Read the data

In [106]:
ls8_df = pandas.read_csv('LS8OLI.rsr', header=0)
s2a = pandas.read_csv('S2AMSI.rsr', header=0)

What column names do we have

In [107]:
ls8_df.columns

Index([u'SR_WL', u'coastal', u'blue', u'green', u'red', u'nir', u'cirrus',
       u'swir1', u'swir2', u'pan'],
      dtype='object')

In [109]:
s2a.columns

Index([u'SR_WL', u'Band 1', u'Band 2', u'Band 3', u'Band 4', u'Band 5',
       u'Band 6', u'Band 7', u'Band 8', u'Band 8A', u'Band 9', u'Band 10',
       u'Band 11', u'Band 12'],
      dtype='object')

# # Coastal band

Calculate upper, lower and centre wavelengths

In [110]:
spline = UnivariateSpline(ls8_df['SR_WL'], ls8_df['coastal'] - ls8_df['coastal'].max()/2, s=0)
target_lower, target_upper = spline.roots()
target_lower, target_upper = spline.roots()
print "Target lower: {}".format(target_lower)
print "Target upper: {}".format(target_upper)
print "Targe centre: {}".format(target_mean_wavelength)

Target lower: 434.935475992
Target upper: 450.900006753
Targe centre: 442.982211078


In [111]:
lower = []
upper = []
centre = []
idx = []
for col in s2a.columns:
    if col == 'SR_WL':
        continue
    spline = UnivariateSpline(s2a['SR_WL'], s2a[col] - s2a[col].max() / 2, s=0)
    xbar = numpy.average(s2a['SR_WL'], weights=s2a[col])
    roots = spline.roots()
    lower.append(roots[0])
    upper.append(roots[1])
    centre.append(xbar)
    idx.append(col)

In [112]:
df = pandas.DataFrame({'lower': lower, 'upper': upper, 'centre':centre}, index=idx)

Using the upper and lower wavelengths, calculate overlaps

In [113]:
df['result'] = (target_lower - df['lower']).abs() + (target_upper - df['upper']).abs()

In [114]:
df

,centre,lower,upper,result
Band 1,443.929445,434.721273,452.668945,1.983141
Band 2,496.541070,469.750194,523.910362,107.825073
Band 3,560.006375,542.681353,577.526008,234.371878
Band 4,664.449162,649.091803,679.732731,442.989051
Band 5,703.886979,697.056647,711.066815,522.287979
Band 6,740.223453,733.372889,747.016050,594.553457
Band 7,782.473512,773.286005,792.222771,679.673293
Band 8,835.110187,782.359328,891.536672,788.060517
Band 8A,864.801258,854.583171,874.894181,843.641869
Band 9,945.027529,935.337343,954.817768,1004.319628


In [115]:
df['result'].argmin()

'Band 1'

In [123]:
print "Target lower: {}".format(target_lower)
print "Target upper: {}".format(target_upper)
print "Targe centre: {}".format(target_mean_wavelength)
print "\nClosest matching result:"
df.loc[df['result'].argmin()]

Target lower: 1363.32892973
Target upper: 1383.72516973
Targe centre: 442.982211078

Closest matching result:


centre      443.929445
lower       434.721273
upper       452.668945
result        1.983141
result2    1859.663882
Name: Band 1, dtype: float64

## Cirrus band

In [117]:
spline = UnivariateSpline(ls8_df['SR_WL'], ls8_df['cirrus'] - ls8_df['cirrus'].max()/2, s=0)
target_lower, target_upper = spline.roots()
target_mean_wavelength = numpy.average(ls8_df['SR_WL'], weights=ls8_df['coastal'])
print "Target lower: {}".format(target_lower)
print "Target upper: {}".format(target_upper)
print "Targe centre: {}".format(target_mean_wavelength)

Target lower: 1363.32892973
Target upper: 1383.72516973
Targe centre: 442.982211078


In [118]:
df['result2'] = (target_lower - df['lower']).abs() + (target_upper - df['upper']).abs()

In [119]:
df

,centre,lower,upper,result,result2
Band 1,443.929445,434.721273,452.668945,1.983141,1859.663882
Band 2,496.541070,469.750194,523.910362,107.825073,1753.393544
Band 3,560.006375,542.681353,577.526008,234.371878,1626.846738
Band 4,664.449162,649.091803,679.732731,442.989051,1418.229565
Band 5,703.886979,697.056647,711.066815,522.287979,1338.930638
Band 6,740.223453,733.372889,747.016050,594.553457,1266.665160
Band 7,782.473512,773.286005,792.222771,679.673293,1181.545323
Band 8,835.110187,782.359328,891.536672,788.060517,1073.158100
Band 8A,864.801258,854.583171,874.894181,843.641869,1017.576748
Band 9,945.027529,935.337343,954.817768,1004.319628,856.898988


In [120]:
df['result2'].argmin()

'Band 10'

In [124]:
print "Target lower: {}".format(target_lower)
print "Target upper: {}".format(target_upper)
print "Targe centre: {}".format(target_mean_wavelength)
print "\nClosest matching result:"
df.loc[df['result2'].argmin()]

Target lower: 1363.32892973
Target upper: 1383.72516973
Targe centre: 442.982211078

Closest matching result:


centre     1373.461883
lower      1358.960898
upper      1388.049700
result     1861.175115
result2       8.692562
Name: Band 10, dtype: float64

## NIR band

In [125]:
spline = UnivariateSpline(ls8_df['SR_WL'], ls8_df['nir'] - ls8_df['nir'].max()/2, s=0)
target_lower, target_upper = spline.roots()
target_mean_wavelength = numpy.average(ls8_df['SR_WL'], weights=ls8_df['coastal'])
print "Target lower: {}".format(target_lower)
print "Target upper: {}".format(target_upper)
print "Targe centre: {}".format(target_mean_wavelength)

Target lower: 850.537132246
Target upper: 878.722304117
Targe centre: 442.982211078


In [126]:
df['result3'] = (target_lower - df['lower']).abs() + (target_upper - df['upper']).abs()

In [127]:
df

,centre,lower,upper,result,result2,result3
Band 1,443.929445,434.721273,452.668945,1.983141,1859.663882,841.869219
Band 2,496.541070,469.750194,523.910362,107.825073,1753.393544,735.598881
Band 3,560.006375,542.681353,577.526008,234.371878,1626.846738,609.052075
Band 4,664.449162,649.091803,679.732731,442.989051,1418.229565,400.434902
Band 5,703.886979,697.056647,711.066815,522.287979,1338.930638,321.135975
Band 6,740.223453,733.372889,747.016050,594.553457,1266.665160,248.870497
Band 7,782.473512,773.286005,792.222771,679.673293,1181.545323,163.750660
Band 8,835.110187,782.359328,891.536672,788.060517,1073.158100,80.992173
Band 8A,864.801258,854.583171,874.894181,843.641869,1017.576748,7.874161
Band 9,945.027529,935.337343,954.817768,1004.319628,856.898988,160.895675


In [129]:
df['result3'].argmin()

'Band 8A'

In [131]:
print "Target lower: {}".format(target_lower)
print "Target upper: {}".format(target_upper)
print "Targe centre: {}".format(target_mean_wavelength)
print "\nClosest matching result:"
df.loc[df['result3'].argmin()]

Target lower: 850.537132246
Target upper: 878.722304117
Targe centre: 442.982211078

Closest matching result:


centre      864.801258
lower       854.583171
upper       874.894181
result      843.641869
result2    1017.576748
result3       7.874161
Name: Band 8A, dtype: float64